In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.gold_serving.dimension_methods
USING DELTA
AS
SELECT
  time,
  sop,
  method_id,
  meth_id,
  temp_id,
  report_template_version,
  seq_id,
  seq_version
FROM workspace.gold_staging.stg_methods;

MERGE INTO workspace.gold_serving.dimension_methods AS target
USING (
  SELECT * FROM (
    SELECT
      time,
      sop,
      method_id,
      meth_id,
      temp_id,
      report_template_version,
      seq_id,
      seq_version,
      ROW_NUMBER() OVER (
        PARTITION BY time, method_id
        ORDER BY time DESC
      ) AS rn
    FROM workspace.gold_staging.stg_methods
  ) WHERE rn = 1  -- Filter duplicates inside the subquery
) AS source
ON target.method_id = source.method_id
AND target.time = source.time
WHEN MATCHED THEN
  UPDATE SET
    target.sop = source.sop,
    target.meth_id = source.meth_id,
    target.temp_id = source.temp_id,
    target.report_template_version = source.report_template_version,
    target.seq_id = source.seq_id,
    target.seq_version = source.seq_version
WHEN NOT MATCHED THEN
  INSERT (
    time,
    sop,
    method_id,
    meth_id,
    temp_id,
    report_template_version,
    seq_id,
    seq_version
  )
  VALUES (
    source.time,
    source.sop,
    source.method_id,
    source.meth_id,
    source.temp_id,
    source.report_template_version,
    source.seq_id,
    source.seq_version
  )
